In [1]:
%run common.ipynb

tokenizer.decode(tokenizer.get_data(third_number=True))

'S-46.62=-15.67-12.66+-18.29E'

In [2]:
model_dpo = torch.load('dpo.model')
model_dpo.to(device)
model_dpo.eval()

/root/anaconda3/envs/pt2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModelGEN(
  (feature): LlamaModel(
    (embed_tokens): Embedding(22, 64)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=64, out_features=64, bias=False)
          (k_proj): Linear(in_features=64, out_features=64, bias=False)
          (v_proj): Linear(in_features=64, out_features=64, bias=False)
          (o_proj): Linear(in_features=64, out_features=64, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=64, out_features=64, bias=False)
          (up_proj): Linear(in_features=64, out_features=64, bias=False)
          (down_proj): Linear(in_features=64, out_features=64, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (fc_out): Linear(in_features=64, out_features=22

In [3]:
#随机一批数据
input_ids = [tokenizer.get_data(third_number=True) for i in range(64)]

#切分成question和answer
split = [i.index(tokenizer.encoder['=']) + 1 for i in input_ids]
question = [input_ids[i][:split[i]] for i in range(len(input_ids))]
answer = [input_ids[i][split[i]:] for i in range(len(input_ids))]

#根据question生成predict
input_ids = [torch.LongTensor(i).unsqueeze(0).to(device) for i in question]
predict = [generate(model_dpo, i) for i in input_ids]

#裁剪,只要生成的部分
predict = [p[0].tolist()[len(q):] for p, q in zip(predict, question)]

#解码成文本
question = [tokenizer.decode(i) for i in question]
answer = [tokenizer.decode(i) for i in answer]
predict = [tokenizer.decode(i) for i in predict]

for q, a, p in zip(question, answer, predict):
    try:
        diff = abs(float(q[1:-1]) - eval(p[:p.index('E')]))
    except:
        diff = abs(float(q[1:-1]))

    print(q, p, diff)

S35.75= -3.68+39.08+0.52E 0.1700000000000017
S129.94= 20.66--89.97+19.86E 0.5500000000000114
S-77.23= 63.49/-14.79+-73.67E 0.7327653820148754
S8346.47= -96.48*-87.66+-49.57E 61.39680000000044
S-38.34= 39.43/97.51+-38.56E 0.1843687826889564
S102.18= 36.68-9.98+76.29E 0.8100000000000023
S-66.36= -5.25-64.38+6.08E 2.8100000000000023
S116.83= 61.96+-21.66+78.47E 1.9399999999999977
S-95.93= -14.80+12.64+-94.56E 0.789999999999992
S-177.68= 6.56-98.00+-83.68E 2.5600000000000023
S2198.58= -39.34*-57.20+-2.46E 49.20800000000054
S3384.91= 79.58*42.10+-21.93E 56.52199999999948
S2333.02= 69.64*33.99+-83.21E 49.16640000000007
S1149.11= -71.22*-14.15+85.16E 56.1869999999999
S63.19= 23.36+34.59+5.27E 0.030000000000001137
S98.02= 87.57+-16.33+26.06E 0.7199999999999989
S90.55= -21.48/45.30+90.15E 0.8741721854304529
S127.17= 84.55+-48.06+91.37E 0.6899999999999977
S-55.52= -61.50/-37.93+-57.72E 0.5785921434220924
S-13.52= 43.77-83.07+26.76E 0.9800000000000111
S-189.71= -88.53+-58.18+-43.45E 0.45000000000